In [1]:
!pip install lxml
!pip install requests
!pip install openpyxl

In [17]:
import requests
headers = {"user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36"}
from lxml import html

#제 20대 국회입법현황 page1-792 url: range(1, 793)
congress_urls = []
for i in range(1,756):
    congress_urls.append("https://opinion.lawmaking.go.kr/gcom/nsmLmSts/out?pageIndex=" + str(i))
    
#제 21대 국회입법현황 page1-792 url : range(1, 793)
for i in range(1,793):
    congress_urls.append("https://opinion.lawmaking.go.kr/gcom/nsmLmSts/out?scBlNm=scBlNm_blNm&sugCd=20&pageIndex=" + str(i))

In [18]:
#발의된 법안과 발의 날짜, 소관부처를 크롤링

law_list = []
for congress_url in congress_urls:
    text_page = requests.get(congress_url, headers=headers)
    content = [i for i in html.fromstring(text_page.text).xpath('//*[@id="listView"]/ul/li[2]/a/text()')]
    initiation_date = [i for i in html.fromstring(text_page.text).xpath('//*[@id="listView"]/ul/li[3]/p/text()')]
    relevant_department = [i for i in html.fromstring(text_page.text).xpath('//*[@id="listView"]/ul/li[4]/p/text()')]
    law_dict = {"CONTENT": content, "INITIATION DATE": initiation_date, "RELEVANT DEPARTMENT": relevant_department}
    law_list.append(law_dict)

#content xpath: //*[@id="listView"]/ul/li[2]/a/text()
#initiation_date xpath: //*[@id="listView"]/ul/li[3]/p/text()
#relevant_department xpath: //*[@id="listView"]/ul/li[4]/p/text()

#//*[@id="listView"]/ul[20]/li[2]/a (ul은 한 page에 1-20까지)

In [16]:
#law_list: 한 dict에 법안과 날짜가 20개씩 들어가게 됨

In [6]:
import pandas as pd

In [35]:
#위 문제를 해결하기 위해서 dataframe으로 만든 뒤, column 별로 list로 만들어줌
congress_law = pd.DataFrame(law_list)
congress_content = congress_law['CONTENT'].tolist()
congress_date = congress_law['INITIATION DATE'].tolist()
congress_department = congress_law['RELEVANT DEPARTMENT'].tolist()

#2차원 리스트의 형태를 1차원으로 
congress_content = sum(congress_content, [])
congress_date = sum(congress_date, [])
congress_department = sum(congress_department, [])

#len(congress_content) #30936
#len(congress_date) #30937
#len(congress_department) #30937

#congress_content에서 missing data 발생 [제 21대 국회입법현황 page 313 (list index 6254)]
del congress_date[6254]
del congress_department[6254]

In [36]:
#list들을 다시 하나의 dataframe으로 만들어줌
congress = pd.DataFrame({"CONTENT": congress_content, "INITIATION DATE": congress_date, "RELEVANT DEPARTMENT": congress_department})

In [13]:
import re

In [37]:
#날짜 크롤링 과정 중에 쓸데없는 정보가 들어갔으므로 정규표현식으로 필요없는 부분을 지워줌
congress['INITIATION DATE'] = congress['INITIATION DATE'].astype(str).str.findall(r'[0-9\.]+')
congress['RELEVANT DEPARTMENT'] = congress['RELEVANT DEPARTMENT'].astype(str).str.findall(r'[ㄱ-ㅣ가-힣]+')

In [38]:
#쪼개져서 list로 존재하던 날짜를 join하여 하나의 문자열로 만듦
for i in range(len(congress.index)):
    congress['INITIATION DATE'][i] = ''.join(congress['INITIATION DATE'][i])
    congress['RELEVANT DEPARTMENT'][i] = ''.join(congress['RELEVANT DEPARTMENT'][i])
congress

,CONTENT,INITIATION DATE,RELEVANT DEPARTMENT
0,국회법 일부개정법률안,2022.5.6.,법제처
1,정부조직법 일부개정법률안,2022.5.6.,행정안전부
2,스토킹범죄의 처벌 등에 관한 법률 일부개정법률안,2022.5.6.,법무부
3,지방자치법 일부개정법률안,2022.5.6.,행정안전부
4,대통령직 인수에 관한 법률 일부개정법률안,2022.5.6.,행정안전부
...,...,...,...
30931,상훈법 일부개정법률안,2017.8.18.,행정안전부
30932,경륜·경정법 일부개정법률안,2017.8.18.,문화체육관광부
30933,도시 및 주거환경정비법 일부개정법률안,2017.8.18.,국토교통부
30934,교통약자의 이동편의 증진법 일부개정법률안,2017.8.18.,국토교통부


In [39]:
#청와대 국민청원 서비스 시작일에 해당하지 않는 2022.8.18 법안 삭제
congress = congress.drop([30929, 30930, 30931, 30932, 30933, 30934, 30935])
congress

,CONTENT,INITIATION DATE,RELEVANT DEPARTMENT
0,국회법 일부개정법률안,2022.5.6.,법제처
1,정부조직법 일부개정법률안,2022.5.6.,행정안전부
2,스토킹범죄의 처벌 등에 관한 법률 일부개정법률안,2022.5.6.,법무부
3,지방자치법 일부개정법률안,2022.5.6.,행정안전부
4,대통령직 인수에 관한 법률 일부개정법률안,2022.5.6.,행정안전부
...,...,...,...
30924,식품위생법 일부개정법률안,2017.8.21.,식품의약품안전처
30925,의료기사 등에 관한 법률 일부개정법률안,2017.8.21.,보건복지부
30926,농어촌주민의 보건복지 증진을 위한 특별법 일부개정법률안,2017.8.21.,보건복지부
30927,사회서비스 이용 및 이용권 관리에 관한 법률 일부개정법률안,2017.8.21.,보건복지부


In [40]:
congress.to_csv("congress_law_crawling.csv", index=False)

In [42]:
congress.to_excel("congress_law_crawling.xlsx", index=False)